In [ ]:
!pip install geemap

This script downloads Land Cover Dynamics products for specific site locations from Google Earth Engine.

1. The script requires a file, "new_site_data.csv" with site locations and dates.
2. The files are downloaded to folders on a Google Drive account. Folder names are 'modis_lsp' and 'viirs_lsp'.
3. LCD data are downloaded as .csv files, with a separate folder for each site, and a separate file for each year at a site.
4. LCD data is downloaded for the years 2017-2024.

In [2]:
import ee
import geemap
import os
import pandas as pd

In [ ]:
Map = geemap.Map()
Map

In [4]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [5]:
sites = pd.read_csv('/content/new_site_data.csv')
sites.head(5)

,Location,new_start,new_end,X,lon,lat,Pod1,Pod2,Pod3,Group,start_date,end_date,Ortho,Scene,order_id_s_h
0,choma_A000127,09/02/2017,03/02/2018,22,27.0922,-16.7795,A000127,NaN,NaN,choma,11/01/2017,05/31/2018,0,85,0d688599-b6bb-47d8-a3cd-a6a8d736ec5b
1,choma_A000127,03/02/2018,07/30/2018,22,27.0922,-16.7795,A000127,NaN,NaN,choma,11/01/2017,05/31/2018,0,85,0d688599-b6bb-47d8-a3cd-a6a8d736ec5b
2,choma_A000212,09/02/2017,03/02/2018,23,26.9253,-16.8294,A000212,NaN,NaN,choma,11/01/2017,05/31/2018,0,2,97cfdbe9-39d1-41ab-8e6e-3d355b598b85
3,choma_A000212,03/02/2018,07/30/2018,23,26.9253,-16.8294,A000212,NaN,NaN,choma,11/01/2017,05/31/2018,0,2,97cfdbe9-39d1-41ab-8e6e-3d355b598b85
4,choma_A000247,09/02/2017,03/02/2018,24,27.1179,-16.7724,A000247,NaN,NaN,choma,11/01/2017,05/31/2018,0,76,95c69caa-67fa-405f-ae2d-0456838c7272


In [6]:
len(sites)

611

Make Google Drive folders

In [7]:
modis_dir = "/content/gdrive/MyDrive/modis_lsp/"
viirs_dir = "/content/gdrive/MyDrive/viirs_lsp/"
if not os.path.exists(modis_dir):
      os.mkdir(modis_dir)
if not os.path.exists(viirs_dir):
      os.mkdir(viirs_dir)

Download MODIS LCD data (MODIS/061/MCD12Q2)

In [ ]:
# def clip_roi(image, roi):
#   return (image.clip(roi).copyProperties(image))

location_list = []

for k in range(2):
  current_site = sites.Location[k]
  print(current_site)
  print(k)
  if current_site in location_list:
    continue
  location_list.append(current_site)
  print(location_list)
  long = sites.lon[k]
  lat = sites.lat[k]
  print(long)
  print(lat)

  for year in range(2017, 2024):
    print(year)


    start_date = str(year) + "-01-01"
    end_date = str(year) + "-01-02"
    point = ee.Geometry.Point(long, lat)
    point_buffer = ee.Geometry.Point(long, lat).buffer(100).bounds()


    #### MODIS LSP
    coll = ee.ImageCollection("MODIS/061/MCD12Q2").filterDate(start_date, end_date)#.filterBounds(point_buffer)  #.map(lambda image: clip_roi(image, point_buffer))
    image_size = coll.size().getInfo()
    if image_size ==0:
      continue
    print('size ' + str(image_size))
    image = coll.first()


      # Load the image you want to extract values from
  # image = ee.Image(YOUR_IMAGE)

    # Reducer function to get all band values at the point of interest
    reducer = ee.Reducer.mean()

    # Use reduceRegion() to get the pixel values at the point of interest
    sampledData = image.reduceRegion(reducer, point, 500).getInfo()

    # Convert the sampledData dictionary to a pandas DataFrame
    df = pd.DataFrame(list(sampledData.items()), columns=['Band', 'Value'])
    print(df)

    outdir = "/content/gdrive/MyDrive/modis_lsp/" + current_site + "/"
    if not os.path.exists(outdir):
      os.mkdir(outdir)
    csv_path = outdir + "modis_lsp_" + str(year) + '.csv'
    df.to_csv(csv_path)




Download VIIRS LCD data (NOAA/VIIRS/001/VNP22Q2)

In [ ]:
 #### VIIRS lCD


location_list = []

for k in range(len(sites)):
  current_site = sites.Location[k]
  print(current_site)
  print(k)
  if current_site in location_list:
    continue
  location_list.append(current_site)
  print(location_list)
  long = sites.lon[k]
  lat = sites.lat[k]
  print(long)
  print(lat)

  for year in range(2017, 2024):
    print(year)


    start_date = str(year) + "-01-01"
    end_date = str(year) + "-01-02"
    point = ee.Geometry.Point(long, lat)
    point_buffer = ee.Geometry.Point(long, lat).buffer(100).bounds()



    coll = ee.ImageCollection("NOAA/VIIRS/001/VNP22Q2").filterDate(start_date, end_date)#.filterBounds(point_buffer)  #.map(lambda image: clip_roi(image, point_buffer))
    image_size = coll.size().getInfo()
    if image_size ==0:
      continue
    print('size ' + str(image_size))
    image = coll.first()


      # Load the image you want to extract values from
  # image = ee.Image(YOUR_IMAGE)

    # Reducer function to get all band values at the point of interest
    reducer = ee.Reducer.mean()

    # Use reduceRegion() to get the pixel values at the point of interest
    sampledData = image.reduceRegion(reducer, point, 500).getInfo()

    # Convert the sampledData dictionary to a pandas DataFrame
    df = pd.DataFrame(list(sampledData.items()), columns=['Band', 'Value'])
    print(df)

    outdir = "/content/gdrive/MyDrive/viirs_lsp/" + current_site + "/"
    if not os.path.exists(outdir):
      os.mkdir(outdir)
    csv_path = outdir + "viirs_lsp_" + str(year) + '.csv'
    df.to_csv(csv_path)
